<b>
<div style="text-align: center;">
    <span style="font-size: 200%;">Sarsa</span>
</div>
</b>

<br>

<b>
<div style="text-align: right;">
    <span style="font-size: 150%;">
        2020/3/19
    </span><br>
    <span style="font-size: 150%;">
        Masaya Mori
    </span>
</div>
</b>

## 目的

<span style="font-size: 120%;"> Sarsaの理論・仕組みを知る．</span> <br>
<span style="font-size: 120%;"> SarsaによるMountainCarの実装を行う．</span>

## Sarsaとは

<span style="font-size: 120%;"> Sarsaとは，[時間的差分学習(Temporal Difference Learning; TD学習)]()を用いた<b>最適方策決定</b>を行うための手法の1つである．</span> <br>

<span style="font-size: 120%;"> TD学習を軽く説明すると，シミュレーションを行うことで状態・行動価値を求めるため，確率遷移がわからない問題に対して適用することができる点(モンテカルロ法の利点)と，ブートストラップを使うことができるため，エピソードの終点まで待たなくても状態・行動価値を更新することができる点(動的計画法の利点)の2つの利点を持つ<u>状態・行動価値関数更新手法</u>である．式で表すと，</span> <br>

<span style="font-size: 120%;">
$$
V(s_t) \leftarrow V(s_t) + \alpha\bigl[r_{t+1} + \gamma \cdot V(s_{t+1}) - V(s_t)\bigr]，\tag{1}
$$
<span><br>

<span style="font-size: 120%;"> と表される．行動価値は式(1)を，</span> <br>

<span style="font-size: 120%;">
\begin{eqnarray}
    Q(s_t,a_t) &\leftarrow& Q(s_t,a_t) + \alpha\bigl[r_{t+1} + \gamma \cdot Q(s_{t+1},a_{t+1}) - Q(s_t,a_t)\bigr]，\tag{2} \\
    Q(s,a) &\leftarrow& Q(s,a) + \alpha\bigl[r' + \gamma \cdot Q(s',a') - Q(s,a)\bigr]，\tag{3}
\end{eqnarray}
<span><br>

<span style="font-size: 120%;"> のように書き換えることで導出することが可能となる．</span> <br>

<span style="font-size: 120%;"> 
Sarsaでは，状態価値より行動価値を求めたほうが都合が良いため，式(3)と$\epsilon$-greedy法を用いることでエピソード(方策)の更新を行う．
具体的には，方策を決定した後にシミュレーション(行動)を行い，式(3)を用いることで行動価値を更新し，最後に<font color="red">$\epsilon$-greedy法</font>によって方策を更新する．
ここで注意すべき点は，<b>方策更新に用いるエピソードは，その時に行ったエピソードである</b>．
</span> <br>

<img src="figure1.jpg" width="900">

<span style="font-size: 120%;"> 上の画像を用いて説明すると，</span> <br>

<span style="font-size: 120%;"> 1. 初めに$\epsilon$-greedy法で方策を決定する．今回は仮に赤線のルートになったとする．</span> <br>
<span style="font-size: 120%;"> 2. 次に行動$a''$で失敗し，状態$S'''$で止まったとする．</span> <br>
<span style="font-size: 120%;"> 3. その時の行動価値の更新は，赤線の行動が保有する行動価値を用いて，赤線の行動が保有する行動価値の更新を行う．</span> <br>
<span style="font-size: 120%;"> 4. 最後に，行動価値が更新されたので再度$\epsilon$-greedy法を用いて方策を決定する．</span> <br>

<span style="font-size: 120%;"> よくわからなかった人は，[Q-Learning]()とSarsaを比較してもらうとわかりやすいかもしれないです．</span> <br>

## MountainCarの実装における注意点

<span style="font-size: 120%;"> 試しに動かしてみる． </span> <br>

In [1]:
# -*- coding:utf-8 -*-
import gym
import time

# 環境の生成（車の山登り）
env = gym.make('MountainCar-v0')
env.reset()

# 環境の描画
for _ in range(200):
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample())
    time.sleep(0.01)
env.close()

<span style="font-size: 120%;"> observation, reward, done, infoの中身を確認してみると， </span> <br>

In [2]:
print("-------------------------------------------")
print("observation = " + str(observation))
print("-------------------------------------------")
print("reward      = " + str(reward))
print("-------------------------------------------")
print("done        = " + str(done))
print("-------------------------------------------")
print("info        = " + str(info))
print("-------------------------------------------")

-------------------------------------------
observation = [-0.4909148  -0.00531235]
-------------------------------------------
reward      = -1.0
-------------------------------------------
done        = True
-------------------------------------------
info        = {'TimeLimit.truncated': True}
-------------------------------------------


<span style="font-size: 120%;"> となっている．この時， </span> <br>

<ul>
<span style="font-size: 120%;"> <li>"observation"はその時の状態の値を保持しており，MountainCarで説明すると，1つ目の要素が車の位置を表しており，2つ目の要素が車の速度を表している．車の位置は[min, max] = [-1.2, 0.6]であり，車の速度は[min, max] = [-0.07, 0.07]となっている．</li></span><br>
<span style="font-size: 120%;"> <li>"reward"は前の行動によって得られた報酬を保持しており，MountainCarでは1ステップ進む毎に-1.0の報酬が与えられるように設計されている．</li></span> <br>
<span style="font-size: 120%;"> <li>"done"は，ゲーム(状態)が続いているか否かをTrue or Falseで保持しており，ゲーム(状態)が続いている場合はTrueを返す．Falseはゲームをクリアした時やゲームに失敗した時など，これ以上ゲームを続けられない時に返される．</li></span> <br>
<span style="font-size: 120%;"> <li>"info"は，デバックする時に役立つ情報を保持している．</li></span><br>
</ul>

<span style="font-size: 120%;"> 詳しい使い方は[こちら](https://gym.openai.com/docs/)と[こちら](https://qiita.com/ishizakiiii/items/75bc2176a1e0b65bdd16#%E5%8F%82%E8%80%83)を参照してください． </span> <br>

<span style="font-size: 120%;"> 今回も『[Cartpole]()』の時と同様に，各状態を連続値から離散値に変換する必要がある．そこで今回は各状態を40個に分割する．また，今回はQ-tableを採用するので，行動は[0, 1, 2] = [左, 何もしない, 右]の3つとなっているため，40×40×3の配列を作成し，そこに行動価値を収納する．(ここから先は合っているかわかりません)行動価値の更新方法はCartpoleの時と同じように行う．詳しくは『[Cartpole]()』の<u>3.1 行動価値関数を用いた方策評価</u>を参照してください．</span> <br>

## Pythonによる実装

### 前準備

In [3]:
import gym
import time
import pickle
import numpy as np
import matplotlib.pyplot as plt

### エージェントクラスの設計

<span style="font-size: 120%;"> エージェントクラスには，『行動』『状態』『報酬』の3つの変数と実際に行動を行う『move関数』を持たせる．また，環境クラスを変えればモンテカルロ法やQ-learningでMountainCarを実装できるように設計する．</span> <br>

In [4]:
class Agent:
    def __init__(self,step_num):
        self.step_num = step_num
        self.actions = [0, 1, 2] # [左, 停止, 右]
        self.states = np.zeros([self.step_num+1,2])
        self.reward = np.zeros(self.step_num)
        self.policy = np.random.choice(self.actions, size=self.step_num, p=[1/3,1/3,1/3])
    
    def get_actions(self):
        return self.actions
    
    def get_policy(self):
        return self.policy
    
    def set_policy(self,policy):
        self.policy = policy
    
    def move(self):
        for i in range(self.step_num):
            self.states[i+1], self.reward[i], done, info = env.step(self.policy[i])
            if done == False :
                return self.states[:i],self.reward[:i],self.policy[:i]
        return self.states[:self.step_num],self.reward[:self.step_num],self.policy[:self.step_num]
            

In [5]:
a = Agent(100)
aa, b, c = a.move()
for i,stte in enumerate(reversed(aa)):
    print(stte)
    print("---------------------")


[-5.94673600e-01 -3.05443339e-04]
---------------------
[-5.94368157e-01  1.67764714e-04]
---------------------
[-0.59453592 -0.00136026]
---------------------
[-0.59317566 -0.0008783 ]
---------------------
[-0.59229736 -0.0023899 ]
---------------------
[-0.58990746 -0.00188394]
---------------------
[-0.58802351 -0.00336413]
---------------------
[-0.58465939 -0.00381952]
---------------------
[-0.58083986 -0.00424673]
---------------------
[-0.57659314 -0.00564251]
---------------------
[-0.57095063 -0.00699646]
---------------------
[-0.56395417 -0.00729838]
---------------------
[-0.55665578 -0.00754591]
---------------------
[-0.54910988 -0.00673705]
---------------------
[-0.54237283 -0.00787778]
---------------------
[-0.53449504 -0.00895949]
---------------------
[-0.52553555 -0.00897402]
---------------------
[-0.51656154 -0.00892124]
---------------------
[-0.5076403 -0.0098016]
---------------------
[-0.4978387  -0.01060859]
---------------------
[-0.48723011 -0.01133637]


In [164]:
aaa = np.array([1,2,3])
bbb = np.array([4,5,6])
ccc = np.array([7,8,9])
print(aaa)
aaa = reversed(aaa)
bbb = reversed(bbb)
ccc = reversed(ccc)
print(aaa)

[1 2 3]


### 環境クラスの設計

<span style="font-size: 120%;"> 環境クラスでは，『状態の離散化』『Q-tableの更新』『$\epsilon$-greedy』『方策の更新』の4つの関数を持たせる．また，Agentクラスを変えればCartpoleなどのさまざまな問題に流用できるように設計する．</span> <br>

In [167]:
class environment:
    def __init__(self,env,actions,division,step_num=200,alpha=0.01,epsilon=0.9):
        # division は分割数をlistで示したものである
        # 例えばCartpoleの場合は[6,6,6,6]であり，今回は[40,40]である
        self.env = env
        self.division = division
        self.q_table = np.zeros(self.division+[len(actions)])
        self.alpha = alpha
        self.epsilon = epsilon
    
    def discretization(self,states):
        # states は[その時に進んだstep数, 状態数]のnp.arrayである
        env_low = self.env.observation_space.low # 位置と速度の最小値
        env_high = self.env.observation_space.high #　位置と速度の最大値
        env_dx = np.array(env_high-env_low) / np.array(self.division) # 分割した時の1マス分の値を算出
        return ((states - env_low) / env_dx).astype(np.int64)
    
    def q_table_update(self,states,rewards,policy):
        for i,(state,reward,action) in enumerate(zip(reversed(states),reversed(reward),reversed(policy))):
            
        
    
    
    

In [119]:
division = [40,40]
print(division)
r = division + [3]
print(r)

[40, 40]
[40, 40, 3]


In [112]:
l = [40,40]
print(l)
# [0, 1, 2]

l.append(3)
print(l)
# [0, 1, 2, 100]

[40, 40]
[40, 40, 3]


In [129]:
env = gym.make('MountainCar-v0')
env.reset()
v = environment(env)
vv = v.discretization(aa,[40,40])
a = np.zeros(env.action_space)
print(a)

TypeError: expected sequence object with len >= 0 or a single integer

## 結果

## 参考資料

[今さら聞けない強化学習（10）: SarsaとQ学習の違い](https://qiita.com/triwave33/items/cae48e492769852aa9f1) <br>
[深層強化学習アルゴリズムまとめ](https://qiita.com/shionhonda/items/ec05aade07b5bea78081) <br>
[OpenAI Gym 入門](https://qiita.com/ishizakiiii/items/75bc2176a1e0b65bdd16#%E5%8F%82%E8%80%83) <br>
[Getting Started with Gym](https://gym.openai.com/docs/) <br>

## メモ

<span style="font-size: 120%;"> Q-learningの更新式はそのルート(行動)での最大の期待値を計算している．</span> <br>
<span style="font-size: 120%;"> つまり，イプシロン・グリーディによって方策を決定した場合，その方策(つまり行動)の現在取りうる最大の価値を計算している．したがって，行動価値を更新するときは方策を使用しない(つまり，イプシロン・グリーディを用いない)が，方策を決めるときは全てのパターンに対してシミュレーションを行わなければならないので，イプシロン・グリーディを用いて方策を決定する．方策オフというのは，更新式に方策(イプシロン・グリーディ)を用いてないからである．</span> <br>
<span style="font-size: 120%;"> Sarsaの更新式は，そのルート(行動)の最大の期待値ではなく，仮定の状態も含まれている．</span> <br>
<span style="font-size: 120%;"> 例えば，釣りで魚を釣り上げることを考えた場合に，Q-learningの場合は，リールを力いっぱい巻けば魚は釣れるので行動価値も高くなるが，Sarsaの場合は，糸が切れることなども考慮されるので，力いっぱい引くのではなく，強弱をつけたりなどの保険要素が付け加えられる．</span> <br>